In [ ]:
# ! pip install --upgrade ultralytics

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ----- ---------------------------------- 0.1/1.1 MB 2.1 MB/s eta 0:00:01
   --------- ------------------------------ 0.3/1.1 MB 2.3 MB/s eta 0:00:01
   ---------------- ----------------------- 0.4/1.1 MB 2.7 MB/s eta 0:00:01
   ------------------------ --------------- 0.7/1.1 MB 3.2 MB/s eta 0:00:01
   ------------------------------- -------- 0.8/1.1 MB 3.3 MB/s eta 0:00:01
   ------------------------------------ --- 1.0/1.1 MB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 3.3 MB/s eta 0:00:00
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.2.2
    Uninstalling ultralytics-8.2.2:
      Successfully uninstalled ultralytics-8.2.2



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
from ultralytics import YOLO

In [25]:
# Load a model
model = YOLO("yolov8m.pt")

In [26]:
# # Use the model
# model.train(data="charDetection-3/data.yaml", epochs=17)  # train the model
# metrics = model.val()  # evaluate model performance on the validation set

In [27]:
import cv2
import numpy as np

# parameter
input_folder = "Satriadata/Test/"
target_size = 640

# Fungsi: letterbox resize (biar tidak merusak aspek rasio)
def letterbox_resize(image, target_size=640):
    h, w = image.shape[:2]
    scale = target_size / max(h, w)
    new_w, new_h = int(w * scale), int(h * scale)
    resized = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

    canvas = np.full((target_size, target_size, 3), 128, dtype=np.uint8)
    top = (target_size - new_h) // 2
    left = (target_size - new_w) // 2
    canvas[top:top+new_h, left:left+new_w] = resized
    return canvas

# Fungsi: CLAHE untuk perbaikan kontras
def apply_clahe(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    l = clahe.apply(l)
    lab = cv2.merge((l, a, b))
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

# Fungsi: Gamma correction (buat gambar lebih terang)
def adjust_gamma(image, gamma=1.2):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

# Fungsi: pertegas warna hitam pada teks
def enhance_black_text(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # threshold adaptif supaya teks hitam lebih pekat
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                   cv2.THRESH_BINARY_INV, 25, 15)
    mask = cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR)
    enhanced = cv2.addWeighted(image, 0.7, mask, 0.3, 0)
    return enhanced

# Fungsi utama preprocessing
def preprocess_image(img):
    img = letterbox_resize(img, target_size)
    img = apply_clahe(img)
    img = adjust_gamma(img, gamma=1.2)
    img = cv2.bilateralFilter(img, 9, 75, 75)  # denoise tanpa hilangkan tepi
    img = enhance_black_text(img)  # pertegas teks hitam
    return img


In [36]:
import pandas as pd
from ultralytics import YOLO
import os
from collections import Counter

# load model hasil training
model = YOLO("runs/detect/train3/weights/best.pt")  # ganti sesuai path model

# baca data dari CSV
df = pd.read_csv("Satriadata/Test/DataTest1.csv", sep=';')   # pastikan kolomnya "Name of File" dan "kunci"
# cari kolom otomatis (case insensitive)
col_name = [c for c in df.columns if "name" in c.lower()][1]
col_kunci = [c for c in df.columns if "kunci" in c.lower()][0]

print("Kolom nama file:", col_name)
print("Kolom kunci    :", col_kunci)

results_list = []

for _, row in df.iterrows():
    img_path = os.path.join("Satriadata/Test/", str(row[col_name]))
    true_plate = str(row[col_kunci])

    if not os.path.exists(img_path):
        print("⚠️ File tidak ditemukan:", img_path)
        continue

     # Prediksi
    results = model.predict(img_path, conf=0.15, verbose=False)  # threshold 0.25 (bisa diatur)
    r = results[0]
    boxes = r.boxes
    xyxy = boxes.xyxy.cpu().numpy()
    cls = boxes.cls.cpu().numpy().astype(int)

    if len(cls) == 0:
        pred_plate = ""
    else:
        # urutkan kiri ke kanan
        sorted_idx = xyxy[:, 0].argsort()
        pred_plate = "".join([model.names[c] for c in cls[sorted_idx]])

    # hitung persentase benar (pakai karakter yang sama, bukan posisi)
    true_len = len(true_plate)
    pred_len = len(pred_plate)

    true_counter = Counter(true_plate)
    pred_counter = Counter(pred_plate)

    # jumlah karakter yang sama (berdasarkan frekuensi terkecil)
    common = true_counter & pred_counter
    correct_chars = sum(common.values())

    # penalti: jika tebakan lebih panjang
    penalty = max(0, pred_len - true_len)

    adjusted_correct = max(0, correct_chars - penalty)
    perc_correct = (adjusted_correct / true_len) * 100 if true_len > 0 else 0

    results_list.append({
        "File": row[col_name],
        "Kunci": true_plate,
        "Deteksi": pred_plate,
        "Persentase Benar": round(perc_correct, 2)
    })


# simpan hasil
results_df = pd.DataFrame(results_list)
print(results_df)
results_df.to_csv("15hasil_prediksi.csv", index=False)

Kolom nama file: Name
Kolom kunci    : kunci
               File      Kunci    Deteksi  Persentase Benar
0     DataTest1.png   AD7034OE   AQ7034OE             87.50
1     DataTest2.png    A9388EX   A98088EL             57.14
2     DataTest3.png      B16TB      B16TB            100.00
3     DataTest4.png   B1661TKZ   B1661TKZ            100.00
4     DataTest5.png  AD3772ABE    AO77ABE             66.67
..              ...        ...        ...               ...
95   DataTest96.png    B1285UL                         0.00
96   DataTest97.png   AB8644PK   AB8644PK            100.00
97   DataTest98.png   AG9718EG  AG9718EEG             87.50
98   DataTest99.png    B1509UN      B509N             71.43
99  DataTest100.png    B1408RX    B1408RX            100.00

[100 rows x 4 columns]


In [37]:
# hitung rata-rata persentase benar dari seluruh data test
avg_score = results_df["Persentase Benar"].mean()

print(f"Rata-rata persentase benar: {avg_score:.2f}%")

Rata-rata persentase benar: 82.15%


In [42]:
# Jalankan evaluasi dengan dataset yang benar
metrics = model.val(data="charDetection-3\data.yaml", split='test')

print("Precision :", metrics.results_dict['metrics/precision(B)'])
print("Recall    :", metrics.results_dict['metrics/recall(B)'])
print("mAP@0.5   :", metrics.results_dict['metrics/mAP50(B)'])
print("mAP@0.5:0.95 :", metrics.results_dict['metrics/mAP50-95(B)'])


Ultralytics 8.3.184  Python-3.9.12 torch-2.2.2+cpu CPU (AMD Ryzen 5 5500U with Radeon Graphics)
val: Fast image access  (ping: 0.10.0 ms, read: 63.925.3 MB/s, size: 10.5 KB)


val: Scanning D:\Kuliah\Semester 8\Portfolio\Portfolio_ObjectDetection_BDC\charDetection-3\test\labels.cache... 35 images, 1 backgrounds, 0 corrupt: 100%|██████████| 35/35 [00:00<?, ?it/s]

WARNING Box and segment counts should be equal, but got len(segments) = 183, len(boxes) = 251. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:08<00:00,  2.78s/it]


                   all         35        251      0.758      0.892      0.899      0.677
                     0         10         10      0.849        0.8      0.948      0.586
                     1         28         38      0.934      0.947      0.968      0.616
                     2         13         14      0.905          1      0.995      0.851
                     3          8          8      0.835          1      0.939      0.796
                     4          7          8      0.752          1      0.955      0.759
                     5          8          9      0.806          1      0.919      0.695
                     6         11         11      0.945          1      0.995      0.675
                     7          7          8      0.873          1      0.995      0.748
                     8         14         17          1       0.83      0.992      0.685
                     9          9          9      0.766          1      0.955      0.646
                     